In [1]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

Successfully connected to SQL database
success


In [2]:
%%sql
-- Create Students Table
CREATE TABLE Students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER
);



-- Insert into Students
INSERT INTO Students (student_id, name, age) VALUES
(1, 'Alice', 20),
(2, 'Bob', 22),
(3, 'Charlie', 21),
(4, 'Diana', 20);

 * sqlite:///question_bank.db
Done.
4 rows affected.


[]

In [3]:
%%sql
-- Create Courses Table
CREATE TABLE Courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT
);

-- Insert into Courses
INSERT INTO Courses (course_id, course_name) VALUES
(101, 'SQL Basics'),
(102, 'Python for Data'),
(103, 'Data Structures');

 * sqlite:///question_bank.db
Done.
3 rows affected.


[]

In [4]:
%%sql
-- Create Enrollments Table
CREATE TABLE Enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    enrollment_date DATE,
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
);

-- Insert into Enrollments
INSERT INTO Enrollments (enrollment_id, student_id, course_id, enrollment_date) VALUES
(1, 1, 101, '2024-01-10'),
(2, 1, 102, '2024-02-15'),
(3, 2, 101, '2024-01-12'),
(4, 3, 103, '2024-03-20'),
(5, 4, 101, '2024-04-01');

 * sqlite:///question_bank.db
Done.
5 rows affected.


[]

In [5]:
%%sql
SELECT *
FROM Students

 * sqlite:///question_bank.db
Done.


student_id,name,age
1,Alice,20
2,Bob,22
3,Charlie,21
4,Diana,20


In [6]:
%%sql
SELECT *
FROM Courses

 * sqlite:///question_bank.db
Done.


course_id,course_name
101,SQL Basics
102,Python for Data
103,Data Structures


In [7]:
%%sql
SELECT *
FROM Enrollments

 * sqlite:///question_bank.db
Done.


enrollment_id,student_id,course_id,enrollment_date
1,1,101,2024-01-10
2,1,102,2024-02-15
3,2,101,2024-01-12
4,3,103,2024-03-20
5,4,101,2024-04-01


### Q.1 Write a query to display the names of all students along with the courses they are enrolled in.

In [8]:
%%sql

SELECT s.name AS student_name,c.course_name
FROM Enrollments e
JOIN Students s ON s.student_id=e.student_id
JOIN Courses c ON e.course_id=c.course_id

 * sqlite:///question_bank.db
Done.


student_name,course_name
Alice,SQL Basics
Alice,Python for Data
Bob,SQL Basics
Charlie,Data Structures
Diana,SQL Basics


### Q.2 List the names of students who are enrolled in the course "SQL Basics".

In [9]:
%%sql

SELECT s.name AS student_name,c.course_name AS Course_name
FROM Enrollments e
JOIN Students s ON s.student_id=e.student_id
JOIN Courses c ON e.course_id=c.course_id
WHERE Course_name = 'SQL Basics'

 * sqlite:///question_bank.db
Done.


student_name,Course_name
Alice,SQL Basics
Bob,SQL Basics
Diana,SQL Basics


### Q.3. Use a CTE to first find all enrollments that happened in March 2024, and then join to get the student name and course name for those enrollments.

In [11]:
%%sql
SELECT *
FROM Enrollments
WHERE strftime('%m', enrollment_date) = '03'

 * sqlite:///question_bank.db
Done.


enrollment_id,student_id,course_id,enrollment_date
4,3,103,2024-03-20


In [14]:
%%sql


-- step 1 - Total enrollments in march 2024 = Output 
-- JOIN
-- step 2 - student name and course name for those enrollments

WITH en AS (SELECT *
            FROM Enrollments
            WHERE strftime('%m', enrollment_date) = '03'
            )
SELECT s.name AS Name,c.course_name AS Course_name,en.enrollment_id
FROM en
JOIN Students s ON s.student_id=en.student_id
JOIN Courses c ON c.course_id=en.course_id;


 * sqlite:///question_bank.db
Done.


Name,Course_name,enrollment_id
Charlie,Data Structures,4


* Always see how Tables relate - if there are 3 tables, most prob, they will be joined 
* Use CTEs for Simplify JOINS
* Again, Pratice more on this

### Q.4 Use a CTE to count how many courses each student is enrolled in, and display their name along with that count.

In [22]:
%%sql
SELECT * FROM Students

 * sqlite:///question_bank.db
Done.


student_id,name,age
1,Alice,20
2,Bob,22
3,Charlie,21
4,Diana,20


In [15]:
%%sql
SELECT * FROM Enrollments

 * sqlite:///question_bank.db
Done.


enrollment_id,student_id,course_id,enrollment_date
1,1,101,2024-01-10
2,1,102,2024-02-15
3,2,101,2024-01-12
4,3,103,2024-03-20
5,4,101,2024-04-01


In [17]:
%%sql
SELECT * FROM COURSES

 * sqlite:///question_bank.db
Done.


course_id,course_name
101,SQL Basics
102,Python for Data
103,Data Structures


In [16]:
%%sql

-- How many - COUNT
-- each student - GROUP BY
-- Columns - name, count of courses
-- JOINS - we need name,

-- Step 1 - count courses for each student


SELECT student_id, COUNT(*) AS Count_of_courses
FROM Enrollments
GROUP BY student_id

 * sqlite:///question_bank.db
Done.


student_id,Count_of_courses
1,2
2,1
3,1
4,1


In [19]:
%%sql


WITH gf AS (SELECT student_id, COUNT(*) AS Count_of_courses
            FROM Enrollments
            GROUP BY student_id
            )
SELECT s.name, Count_of_Courses
FROM gf
JOIN Students s 
ON gf.student_id=s.student_id

-- This is the most effective

 * sqlite:///question_bank.db
Done.


name,Count_of_courses
Alice,2
Bob,1
Charlie,1
Diana,1


In [20]:
%%sql

SELECT s.name,COUNT(e.course_id) AS Course_Count
FROM Students s
JOIN Enrollments e
ON s.student_id=e.student_id
GROUP BY s.student_id,s.name;

 * sqlite:///question_bank.db
Done.


name,Course_Count
Alice,2
Bob,1
Charlie,1
Diana,1


### Q.5 - Write a query using a CTE to list students who are enrolled in more than one course.

In [21]:
%%sql

-- Students enrolled in more than one course
-- Student_name, count of courses

WITH gf AS (SELECT student_id, COUNT(*) AS Count_of_courses
            FROM Enrollments
            GROUP BY student_id
            )
SELECT s.name, Count_of_Courses
FROM gf
JOIN Students s 
ON gf.student_id=s.student_id
WHERE Count_of_Courses > 1;

 * sqlite:///question_bank.db
Done.


name,Count_of_courses
Alice,2
